In [1]:
_author_ = "Rahul_Chowdhury"
import pandas as pd
import numpy as np
import sys
from collections import defaultdict
print(sys.version)


3.5.2 |Anaconda 4.2.0 (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]


In [2]:
target_url = ("https://archive.ics.uci.edu/ml/machine-learning-databases/tae/tae.data")
Data = pd.read_csv(target_url,header=None, prefix = "V")
k = len(list(Data['V5'].unique())) #counting the number of classes in the last column

Data = Data.drop(Data.columns[[5]], axis=1) #removing 1st and last column
Data.head(10)



,V0,V1,V2,V3,V4
0,1,23,3,1,19
1,2,15,3,1,17
2,1,23,3,2,49
3,1,5,2,2,33
4,2,7,11,2,55
5,2,23,3,1,20
6,2,9,5,2,19
7,2,10,3,2,27
8,1,22,3,1,58
9,2,15,3,1,20


In [3]:
neighborhood_class_dict = {}
threshold = 5
for index, row in Data.iterrows():
    neighbor_list = []
    for index1, row1 in Data.iterrows():
        if((row['V4'] > row1['V4'])):
            if(row['V4'] - row1['V4'] <= threshold):
                neighbor_list.append(index1)
        elif(row['V4'] < row1['V4']):
            if(row1['V4'] - row['V4'] <= threshold):
                neighbor_list.append(index1)
        else:        
            neighbor_list.append(index1)
    neighborhood_class_dict[index] = neighbor_list         
#print(neighborhood_class_dict)
for index in neighborhood_class_dict:
    print("key=", index)
    print(neighborhood_class_dict[index])

key= 0
[0, 1, 5, 6, 9, 21, 23, 26, 31, 36, 39, 40, 44, 45, 48, 60, 62, 65, 70, 75, 79, 83, 85, 86, 87, 89, 90, 91, 95, 96, 98, 99, 101, 102, 104, 107, 110, 115, 119, 121, 123, 129, 130, 132, 133, 134, 135, 139, 142, 145]
key= 1
[0, 1, 5, 6, 9, 23, 25, 31, 33, 36, 39, 40, 44, 45, 48, 62, 64, 70, 72, 75, 79, 83, 85, 86, 87, 89, 90, 95, 96, 98, 99, 101, 102, 104, 107, 110, 117, 119, 121, 123, 129, 130, 132, 133, 134, 139, 142, 147]
key= 2
[2, 17, 30, 37, 41, 56, 69, 76, 84, 125, 127, 131, 148, 149]
key= 3
[3, 11, 12, 22, 24, 27, 29, 32, 34, 35, 38, 42, 50, 51, 61, 63, 66, 68, 71, 73, 74, 77, 88, 93, 94, 100, 103, 108, 109, 111, 112, 114, 118, 124, 126, 138, 141, 143]
key= 4
[4, 8, 30, 43, 47, 69, 125, 149]
key= 5
[0, 1, 5, 6, 9, 21, 23, 26, 31, 36, 39, 40, 44, 45, 48, 60, 62, 65, 70, 75, 78, 79, 83, 85, 86, 87, 89, 90, 91, 92, 95, 96, 99, 101, 102, 104, 105, 110, 115, 119, 121, 123, 129, 130, 132, 133, 134, 135, 139, 142, 145]
key= 6
[0, 1, 5, 6, 9, 21, 23, 26, 31, 36, 39, 40, 44, 45, 48,

In [33]:
cluster1_numerical_global = [] #global variable
cluster2_numerical_global = [] #global variable

def MMeR(U,k):
    C =  {}#the clusters made through MMeR alogrithm
    C_distance = {}
    #Rough_Data = U
    Rough_Data = U.copy()
    #print(Rough_Data)
    index = 0
   
    while(k>len(C.keys())):
        #print(Rough_Data) 
        split_attr,min_roughness_attr_value = mmer_roughness(Rough_Data)
        print(split_attr)
        #min_roughness_attr_value = min(roughness_list_dict[split_attr]) #find the min. value of min_roughness_attr for which alpha is minimum
        #print(min_roughness_attr_value)
        if(split_attr == 'V4'):
            cluster1 = cluster1_numerical_global
            #print("split-cluster1",cluster1 )
            cluster2 = cluster2_numerical_global
            #print("split-cluster2",cluster2 )
        else:    
            cluster1 = Rough_Data[Rough_Data[split_attr] == min_roughness_attr_value].index.tolist()
            cluster2 = Rough_Data.loc[Rough_Data[split_attr] != min_roughness_attr_value].index.tolist()
            
        cluster1_distance = cluster_distance(cluster1, Data) #calculating the cluster distance of cluster1
        cluster2_distance = cluster_distance(cluster2, Data) #calculating the cluster distance of cluster2
        C[index] = cluster1
        C_distance[index] = cluster1_distance
        index += 1
        C[index] = cluster2
        C_distance[index] = cluster2_distance
        index += 1
        max_index = max(C_distance, key= C_distance.get) #find the index with the max cluster distance
        
              
        Rough_Data = U.ix[C[max_index]]  #send data of cluster with max distance
        for key in C:
            print("Cluster",C[key])
            print("Cluster distance", C_distance[key])
      
        
        if(len(C.keys())!= (k-1)):
               
            del C[max_index]
            del C_distance[max_index]
        
        print("C length:",len(C.keys()))
       
        
        
    


    
    
def roughness(col_name1, col_name2, val, Data):
     
     equiv_class_dict = {} #dictionary storing equiv classes for all attribute
     for col in Data:
        equiv_class_dict[col] =list(Data.groupby([col]))
        
     arr = equiv_class_dict[col_name2] #group equiv classes according to column2
     target_set = Data[Data[col_name1] == val].index.tolist() #calculating target set for each unique value in column1
     lower_approx = []
     upper_approx = []
     for name,group in arr:
       selected_list = list(group.index)
       if(set(selected_list).issubset(target_set)): #if a subset, then append to lower_approx
            lower_approx.append(selected_list)
       if(set(selected_list) & set(target_set)): #if intersection is not null, then append to upper_approx
            upper_approx.append(selected_list)
     lower_approx_count = sum(map(len, lower_approx)) #count the no. of elements in lower_approx
     upper_approx_count = sum(map(len, upper_approx))
     if(lower_approx_count == 0):
        roughness = 1
     else:   
         roughness = 1 - (lower_approx_count/upper_approx_count) # return roughness for a(i) = alpha
     return roughness

def roughness_categorical_to_numerical(neighborhood_class_dict, col_name1, col_name2, val, Data):
   
    target_set = Data[Data[col_name1] == val].index.tolist() #calculating target set for each unique value in column1
    lower_approx1 = []
    upper_approx1= []
    for k in neighborhood_class_dict:
       selected_list = neighborhood_class_dict[k]
       if(set(selected_list).issubset(target_set)): #if a subset, then append to lower_approx
            lower_approx1.append(k)
       if(set(selected_list) & set(target_set)): #if intersection is not null, then append to upper_approx
            upper_approx1.append(k)
    lower_approx_count = len(lower_approx1) #count the no. of elements in lower_approx
    upper_approx_count = len(upper_approx1)
    if(lower_approx_count == 0):
        roughness = 1
    else:   
         roughness = 1 - (lower_approx_count/upper_approx_count) # return roughness for a(i) = alpha
    return roughness

def roughness_numerical_to_categorical(neighborhood , col2_name2 , Data):
    
    target_set = neighborhood #target set will be neighbourhood of col1
    equiv_class_dict = {} #dictionary storing equiv classes for all attribute
    for col in Data:
        equiv_class_dict[col] =list(Data.groupby([col]))
        
    arr = equiv_class_dict[col2_name2] #group equiv classes according to column2
    lower_approx = []
    upper_approx = []
    for name,group in arr:
       selected_list = list(group.index)
       if(set(selected_list).issubset(target_set)): #if a subset, then append to lower_approx
            lower_approx.append(selected_list)
       if(set(selected_list) & set(target_set)): #if intersection is not null, then append to upper_approx
            upper_approx.append(selected_list)
    lower_approx_count = sum(map(len, lower_approx)) #count the no. of elements in lower_approx
    upper_approx_count = sum(map(len, upper_approx))
    if(lower_approx_count == 0):
        roughness = 1
    else:   
         roughness = 1 - (lower_approx_count/upper_approx_count) # return roughness for a(i) = alpha
    #print("roughness of numerical", roughness)
    return roughness
   
    
    
    


def cluster_distance(cluster, Data):
        distance = 0
        print("Cluster", cluster)
        Categorical_Data = Data.ix[cluster]
        Numerical_Data = Categorical_Data[['V4']]
        Categorical_Data = Categorical_Data.drop(Categorical_Data.columns[[4]], axis = 1)
        
        
        for i in range(0,len(cluster)-1): #categorical
              a = np.array(Categorical_Data.ix[cluster[i]])
              for k in range(i+1,len(cluster)):
                   b = np.array(Categorical_Data.ix[cluster[k]])
                   distance += sum(a != b)
                
        neighborhood_class_dict = {} #calculating neighborhood of numerical attr
        threshold = 15
        for index, row in Data.iterrows():
                        neighbor_list = []
                        for index1, row1 in Data.iterrows():
                            if((row['V4'] > row1['V4'])):
                                if(row['V4'] - row1['V4'] <= threshold):
                                    neighbor_list.append(index1)
                            elif(row['V4'] < row1['V4']):
                                if(row1['V4'] - row['V4'] <= threshold):
                                    neighbor_list.append(index1)
                            else:        
                                neighbor_list.append(index1)
                        neighborhood_class_dict[index] = neighbor_list
                        
        for i in range(0,len(cluster)-1): #for numerical
              a = neighborhood_class_dict[i]
              for k in range(i+1,len(cluster)):
                b = neighborhood_class_dict[k]
                if(a!= b):
                    distance += 1
        
                
        print(len(cluster))  
        final_distance = (2 * distance)/(len(cluster) * (len(cluster)-1)) 
                  
        return final_distance

def cluster_numerical(cluster1 , cluster2):
    global cluster1_numerical_global 
    cluster1_numerical_global = cluster1
    global cluster2_numerical_global
    cluster2_numerical_global = cluster2
    #print("cluster1_numerical_global", cluster1_numerical_global)
    #print("cluster2_numerical_global", cluster2_numerical_global)
   
    

    
def mmer_roughness(Data):
    #check if all columns have more than 1 unique values, if not drop that column/columns
    #print("Dropped columns:")
    #print(Data)
    for col1 in Data:
         values_in_col_name1 =  list(Data[col1].unique()) #extracting each unique value in column1
         #print(col1, len(values_in_col_name1))
         if(len(values_in_col_name1) == 1):
            Data = Data.drop(col1, axis = 1)
            
   
    
    min_mean_roughness = {} #array to store the minimum of average of roughness of each attr
    roughness_list_dict = defaultdict(dict) #needed to use 2D dictionary 
   
    
    
    for col1 in Data:
        
        neighborhood_class_dict = {} #calculating neighborhood of numerical attr
        threshold = 15
        for index, row in Data.iterrows():
                        neighbor_list = []
                        for index1, row1 in Data.iterrows():
                            if((row['V4'] > row1['V4'])):
                                if(row['V4'] - row1['V4'] <= threshold):
                                    neighbor_list.append(index1)
                            elif(row['V4'] < row1['V4']):
                                if(row1['V4'] - row['V4'] <= threshold):
                                    neighbor_list.append(index1)
                            else:        
                                neighbor_list.append(index1)
                        neighborhood_class_dict[index] = neighbor_list
        
        if(col1 == 'V4'):
            
             mean_roughness_numerical = [] 
             roughness_list_dict_numerical = {}
             for key in neighborhood_class_dict:             
                 roughness_list_numerical = [] 
                 for col2 in Data:
                    if(col1!= col2):   
                       roughness_list_numerical.append(roughness_numerical_to_categorical(neighborhood_class_dict[k], col2, Data))
                 mean_roughness_numerical.append(sum(roughness_list_numerical)/float(len(roughness_list_numerical)))
                 roughness_list_dict_numerical[key] = sum(roughness_list_numerical)/float(len(roughness_list_numerical))
                                  
             min_mean_roughness['V4'] = (min(mean_roughness_numerical))   
             min_roughness_obj =   min(roughness_list_dict_numerical, key= roughness_list_dict_numerical.get)
             l = Data.index.tolist()   
             cluster1 =  neighborhood_class_dict[min_roughness_obj]
             cluster2 =  list(set(l) - set(neighborhood_class_dict[min_roughness_obj]))
             #print("cluster1 of v4", cluster1)
             #print("cluster2 of v4", cluster2)   
                
                
        values_in_col_name1 =  list(Data[col1].unique()) #extracting each unique value in column1
        mean_roughness = [] #list for mean roughness for a(i) = alpha
        for val in values_in_col_name1:
            roughness_list = [] #array of a(i) = alpha w.r.t. other columns 
            for col2 in Data:
                if(col1!= col2): #not to be compared with itself
                  if(col2 == 'V4'):
                      roughness_list.append(roughness_categorical_to_numerical(neighborhood_class_dict, col1 , col2, val, Data))
                                     
                  else:  
                      roughness_list.append(roughness(col1 , col2, val, Data))
            mean_roughness.append(sum(roughness_list)/float(len(roughness_list)))
            roughness_list_dict[col1][val] = sum(roughness_list)/float(len(roughness_list))
        min_mean_roughness[col1]=(min(mean_roughness))
    for key in min_mean_roughness:
        print(key, min_mean_roughness[key]) 
    #print(min_mean_roughness)    
    min_roughness_attr = min(min_mean_roughness, key=min_mean_roughness.get) #find the attr with minimum roughness
    print(min_roughness_attr)
    if(min_roughness_attr == 'V4'):
        cluster_numerical(cluster1, cluster2)
        return 'V4', 1
    
    return min_roughness_attr, min(roughness_list_dict[min_roughness_attr])
     

    


MMeR(Data,k) 













    














V4 0.75
V3 0.7036423841059603
V1 0.75
V2 0.75
V0 0.8360927152317881
V3
V3
Cluster [0, 1, 5, 8, 9, 18, 23, 33, 39, 40, 44, 48, 57, 62, 72, 78, 79, 82, 85, 86, 87, 101, 102]
23
Cluster [2, 3, 4, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 41, 42, 43, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 73, 74, 75, 76, 77, 80, 81, 83, 84, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150]
128
Cluster [0, 1, 5, 8, 9, 18, 23, 33, 39, 40, 44, 48, 57, 62, 72, 78, 79, 82, 85, 86, 87, 101, 102]
Cluster distance 2.49407114625
Cluster [2, 3, 4, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38,